# Installation

In [ ]:
! pip install flordb

# Getting Started

We start by selecting (or creating) a `git` repository to save our model training code as we iterate and experiment. Flor automatically commits your changes on every run, so no change is lost. Below we provide a sample repository you can use to follow along:

In [ ]:
!git clone git@github.com:ucbepic/ml_tutorial ../ml_tutorial

In [2]:
import os
os.chdir('../ml_tutorial/')

Run the `train.py` script to train a small linear model, 
and test your `flordb` installation.

In [ ]:
! python train.py --flor myFirstRun

Flor will manage checkpoints, logs, command-line arguments, code changes, and other experiment metadata on each run (More details [below](#storage--data-layout)). All of this data is then expesed to the user via SQL or Pandas queries.


# View your experiment history
From the same directory you ran the examples above, open an iPython terminal, then load and pivot the log records.


In [3]:
from flor import full_pivot, log_records
df = full_pivot(log_records())

df.head()

STEPPING IN b6208b62f23ff3312364f3d1a3a7ebba0425be47


,projid,runid,tstamp,vid,epoch,step,loss,epochs,lr,hidden,batch_size
0,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,100,0.4003249704837799,5,0.001,500,32
1,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,200,0.3539811670780182,5,0.001,500,32
2,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,300,0.49929293990135193,5,0.001,500,32
3,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,400,0.23701156675815582,5,0.001,500,32
4,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,500,0.16784034669399261,5,0.001,500,32


# Run some more experiments
The `train.py` script has been prepared in advance to define and manage four different hyper-parameters:

In [4]:
%cat train.py | grep flor.arg

hidden_size = flor.arg("hidden", default=500)
num_epochs = flor.arg("epochs", 5)
batch_size = flor.arg("batch_size", 32)
learning_rate = flor.arg("lr", 1e-3)


You can control any of the hyper-parameters (e.g. `hidden`) using Flor's command-line interface:

In [6]:
! python train.py --flor mySecondRun --hidden 75

Epoch [1/5], Step [100/1875], Loss: 0.7720
Epoch [1/5], Step [200/1875], Loss: 0.5216
Epoch [1/5], Step [300/1875], Loss: 0.2941
Epoch [1/5], Step [400/1875], Loss: 0.3155
Epoch [1/5], Step [500/1875], Loss: 0.5852
Epoch [1/5], Step [600/1875], Loss: 0.3453
Epoch [1/5], Step [700/1875], Loss: 0.1697
Epoch [1/5], Step [800/1875], Loss: 0.2625
Epoch [1/5], Step [900/1875], Loss: 0.2256
Epoch [1/5], Step [1000/1875], Loss: 0.3984
Epoch [1/5], Step [1100/1875], Loss: 0.6364
Epoch [1/5], Step [1200/1875], Loss: 0.3515
Epoch [1/5], Step [1300/1875], Loss: 0.1398
Epoch [1/5], Step [1400/1875], Loss: 0.1809
Epoch [1/5], Step [1500/1875], Loss: 0.2526
Epoch [1/5], Step [1600/1875], Loss: 0.0254
Epoch [1/5], Step [1700/1875], Loss: 0.0561
Epoch [1/5], Step [1800/1875], Loss: 0.0543
Epoch [2/5], Step [100/1875], Loss: 0.0632
Epoch [2/5], Step [200/1875], Loss: 0.2124
Epoch [2/5], Step [300/1875], Loss: 0.1795
Epoch [2/5], Step [400/1875], Loss: 0.1249
Epoch [2/5], Step [500/1875], Loss: 0.0871
Ep

### Advanced (Optional): Batch Processing
Alternatively, we can call `flor.batch()` from an interactive environment
inside our model training repository, to dispatch a group of jobs that can be long-runnning:

In [7]:
import flor

jobs = flor.cross_prod(hidden=[i*100 for i in range(1,6)],lr=(1e-4, 1e-3))
assert jobs is not None

flor.batch(jobs)

--hidden 100 --lr 0.0001 
--hidden 100 --lr 0.001 
--hidden 200 --lr 0.0001 
--hidden 200 --lr 0.001 
--hidden 300 --lr 0.0001 
--hidden 300 --lr 0.001 
--hidden 400 --lr 0.0001 
--hidden 400 --lr 0.001 
--hidden 500 --lr 0.0001 
--hidden 500 --lr 0.001 


Then, using a new console or terminal, we start a `flordb` server to process the batch jobs:
```bash
$ python -m flor serve
```

or, if we want to allocate a GPU to the flor server:
```bash
$ python -m flor serve 0 
```
(where 0 is replaced by the GPU id).

You can check the progress of your jobs with the following query:

In [13]:
!sqlite3 ~/.flor/main.db -header 'select done, path, count(*) from jobs group by done, path;'

done|path|count(*)
1|/Users/rogarcia/git/ml_tutorial|10


When finished, the query will report 10 jobs marked as `done` = 1

```
done|path|count(*)
1|/Users/rogarcia/git/ml_tutorial|10
```

You can view the updated pivot view as follows:

In [15]:
df = full_pivot(log_records())
df.head()

,projid,runid,tstamp,vid,epoch,step,loss,epochs,lr,hidden,batch_size
0,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,100,0.4003249704837799,5,0.001,500,32
1,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,200,0.3539811670780182,5,0.001,500,32
2,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,300,0.49929293990135193,5,0.001,500,32
3,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,400,0.23701156675815582,5,0.001,500,32
4,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,500,0.16784034669399261,5,0.001,500,32


# Model Traing Kit (MTK)
The Model Training Kit (MTK) includes utilities for serializing and checkpointing PyTorch state,
and utilities for resuming, auto-parallelizing, and memoizing executions from checkpoint.

In this context, `Flor` is an alias for `MTK`. The model developer passes objects for checkpointing to `Flor.checkpoints(*args)`,
and gives it control over loop iterators by 
calling `Flor.loop(iterator)` as follows:

In [31]:
!cat train.py | grep -B 3 -A 25 Flor.checkpoints 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Flor.checkpoints(model, optimizer)

# Train the model
total_step = len(train_loader)
for epoch in Flor.loop(range(num_epochs)):
    for i, (images, labels) in Flor.loop(enumerate(train_loader)):
        # Move tensors to the configured device
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(
                "Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}".format(
                    epoch + 1,
                    num_epochs,
                    i + 1,
                    total_step,


As shown, 
we wrap both the nested training loop and main loop with `Flor.loop` so Flor can manage their state. Flor will use loop iteration boundaries to store selected checkpoints adaptively, and on replay time use those same checkpoints to resume training from the appropriate epoch.  


### Logging API

You call `flor.log(name, value)` and `flor.arg(name, default=None)` to log metrics and register tune-able hyper-parameters, respectively. 

In [32]:
%cat train.py | grep flor.arg

hidden_size = flor.arg("hidden", default=500)
num_epochs = flor.arg("epochs", 5)
batch_size = flor.arg("batch_size", 32)
learning_rate = flor.arg("lr", 1e-3)


In [33]:
%cat train.py | grep flor.log

                    flor.log("loss", loss.item()),


The `name`(s) you use for the variables you intercept with `flor.log` and `flor.arg` will become a column (measure) in the full pivoted view (see [Viewing your exp history](#view-your-experiment-history)).


# Storage & Data Layout
On each run, Flor will:
1. Save model checkpoints in `~/.flor/`
1. Commit code changes, command-line args, and log records to `git`, inside a dedicated `flor.shadow` branch.

In [46]:
! ls -lagh ~/.flor

total 424
drwxr-xr-x   5 staff   160B Jul 20 12:20 .
drwxr-x---+ 49 staff   1.5K Jul 20 12:19 ..
-rw-r--r--   1 staff    20K Jul 20 12:19 main.db
-rw-r--r--   1 staff   176K Jul 20 12:20 ml_tutorial.db
drwxr-xr-x  30 staff   960B Jul 20 12:21 ml_tutorial_flor.shadow.readme


In [48]:
! echo $(pwd)
! git branch

/Users/rogarcia/git/ml_tutorial


* flor.shadow.readme


In [45]:
! echo $(pwd)'/.flor'
! ls -lagh ./.flor/

/Users/rogarcia/git/ml_tutorial/.flor


total 24
drwxr-xr-x  5 staff   160B Jul 20 12:20 .
drwxr-xr-x  9 staff   288B Jul 20 11:51 ..
-rw-r--r--  1 staff   236B Jul 20 12:20 .replay.json
-rw-r--r--  1 staff   2.8K Jul 20 12:20 log_records.csv
-rw-r--r--  1 staff   225B Jul 20 12:20 seconds.json


Flor will access and interpret contents of `.flor` automatically. The data and log records will be exposed to the user via SQL or Pandas queries.

# Hindsight Logging


Suppose you wanted to start logging the `device`
identifier where the model is run, as well as the
final `accuracy` after training.
You would add the corresponding logging statements
to `train.py`, for example:

In [52]:
%cat train.py | grep -C 4 flor.log

from flor import MTK as Flor

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
flor.log("device", str(device))

# Hyper-parameters
input_size = 784
hidden_size = flor.arg("hidden", default=500)
--
                    epoch + 1,
                    num_epochs,
                    i + 1,
                    total_step,
                    flor.log("loss", loss.item()),
                )
            )

# Test the model
--
        correct += (predicted == labels).sum().item()

    print(
        "Accuracy of the network on the 10000 test images: {} %".format(
            flor.log("accuracy", 100 * correct / total)
        )
    )


In [55]:
! echo $(pwd)
! echo $(git branch)
! git commit -am "hindsight logging stmts added."

/Users/rogarcia/git/ml_tutorial


* flor.shadow.readme
[flor.shadow.readme 048e4aa] hindsight logging stmts added.
 1 file changed, 2 insertions(+), 1 deletion(-)


Typically, when you add a logging statement, logging 
begins "from now on", and you have no visibility into the past.
With hindsight logging, the aim is to allow model developers to send
new logging statements back in time, and replay the past 
efficiently from checkpoint.

In order to do that, we open up an interactive environent from within the `ml_tutorial` directory, and call `flor.replay()`, asking flor to apply the logging statements with the names `device` and `accuracy` to all previous versions (leave `where_clause` null in `flor.replay()`):

In [57]:
flor.replay(['device', 'accuracy'])

,projid,runid,tstamp,vid,prep_secs,eval_secs
0,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,0.070348,0.263011
1,ml_tutorial_flor.shadow.readme,mySecondRun,2023-07-20T12:08:59,1b879cb426e79a472abf5e411c7e755ece63afb5,0.053751,0.214642
2,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:14:43,db531abbb7b835e95144e9def50aa3c8972ca661,0.053085,0.219996
3,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:14:55,f41341c7fce94fba1b2e18954c2ee51018d4d623,0.039456,0.236528
4,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:15:07,bef4d8266294bf84cd5328d4b34dbd9bac9d0cd8,0.041527,0.227430
5,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:15:20,0e3d04cca91e14f785aad26e58c67b90d858dee5,0.038435,0.231740
6,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:15:34,9a58e168c7519fa1f9641647ad16d2df9b38f3b3,0.042505,0.269844
7,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:15:50,41709e4b3445f5dce7a6073df04128448be634df,0.040544,0.243500
8,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:16:05,6f80a222ee2594e5fe63dc9d8d97699d3e17b235,0.038520,0.241692
9,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:16:21,edc9f4866855e7f7e3dd3d8a78d68ef1ff126e4c,0.036969,0.235917


Flordb registered 12 replay jobs.


Then, using a new console or terminal, we start a `flordb` server to process the batch jobs:
```bash
$ python -m flor serve
```

or, if we want to allocate a GPU to the flor server:
```bash
$ python -m flor serve 0 
```
(where 0 is replaced by the GPU id).

You can check the progress of your jobs with the following query:

In [61]:
!sqlite3 ~/.flor/main.db -header 'select done, path, appvars, count(*) from replay group by done, path, appvars;'

done|path|appvars|count(*)
1|/Users/rogarcia/git/ml_tutorial|device, accuracy|24


When the process is finished, you will be able to view the values for `device` and `accuracy` for historical executions, and they will continue to be logged in subsequent iterations:

In [65]:
from flor import full_pivot, log_records
df = full_pivot(log_records())
df.head()

,projid,runid,tstamp,vid,epoch,step,loss,epochs,accuracy,lr,hidden,batch_size,device
0,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,100,0.4003249704837799,5,97.87,0.001,500,32,cpu
1,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,200,0.3539811670780182,5,97.87,0.001,500,32,cpu
2,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,300,0.49929293990135193,5,97.87,0.001,500,32,cpu
3,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,400,0.23701156675815582,5,97.87,0.001,500,32,cpu
4,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,1,500,0.16784034669399261,5,97.87,0.001,500,32,cpu


In [67]:
df[list(flor.DATA_PREP) + ['device', 'accuracy']].drop_duplicates()

,projid,runid,tstamp,vid,device,accuracy
0,ml_tutorial_flor.shadow.readme,myFirstRun,2023-07-20T11:51:45,b6208b62f23ff3312364f3d1a3a7ebba0425be47,cpu,97.87
90,ml_tutorial_flor.shadow.readme,mySecondRun,2023-07-20T12:08:59,1b879cb426e79a472abf5e411c7e755ece63afb5,cpu,97.5
180,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:14:43,db531abbb7b835e95144e9def50aa3c8972ca661,cpu,93.96
270,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:14:55,f41341c7fce94fba1b2e18954c2ee51018d4d623,cpu,97.29
360,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:15:07,bef4d8266294bf84cd5328d4b34dbd9bac9d0cd8,cpu,95.07
450,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:15:20,0e3d04cca91e14f785aad26e58c67b90d858dee5,cpu,97.58
540,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:15:34,9a58e168c7519fa1f9641647ad16d2df9b38f3b3,cpu,95.37
630,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:15:50,41709e4b3445f5dce7a6073df04128448be634df,cpu,97.84
720,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:16:05,6f80a222ee2594e5fe63dc9d8d97699d3e17b235,cpu,95.9
810,ml_tutorial_flor.shadow.readme,BATCH,2023-07-20T12:16:21,edc9f4866855e7f7e3dd3d8a78d68ef1ff126e4c,cpu,97.83


Note the new columns `device` and `accuracy` that are backfilled.

## Publications

To cite this work, please refer to the [Hindsight Logging](http://www.vldb.org/pvldb/vol14/p682-garcia.pdf) paper (VLDB '21).

FLOR is open source software developed at UC Berkeley. 
[Joe Hellerstein](https://dsf.berkeley.edu/jmh/) (databases), [Joey Gonzalez](http://people.eecs.berkeley.edu/~jegonzal/) (machine learning), and [Koushik Sen](https://people.eecs.berkeley.edu/~ksen) (programming languages) 
are the primary faculty members leading this work.

This work is released as part of [Rolando Garcia](https://rlnsanz.github.io/)'s doctoral dissertation at UC Berkeley,
and has been the subject of study by Eric Liu and Anusha Dandamudi, 
both of whom completed their master's theses on FLOR.
Our list of publications are reproduced below.
Finally, we thank [Vikram Sreekanti](https://www.vikrams.io/), [Dan Crankshaw](https://dancrankshaw.com/), and [Neeraja Yadwadkar](https://cs.stanford.edu/~neeraja/) for guidance, comments, and advice.
[Bobby Yan](https://bobbyy.org/) was instrumental in the development of FLOR and its corresponding experimental evaluation.

* [Hindsight Logging for Model Training](http://www.vldb.org/pvldb/vol14/p682-garcia.pdf). _R Garcia, E Liu, V Sreekanti, B Yan, A Dandamudi, JE Gonzalez, JM Hellerstein, K Sen_. The VLDB Journal, 2021.
* [Fast Low-Overhead Logging Extending Time](https://www2.eecs.berkeley.edu/Pubs/TechRpts/2021/EECS-2021-117.html). _A Dandamudi_. EECS Department, UC Berkeley Technical Report, 2021.
* [Low Overhead Materialization with FLOR](https://www2.eecs.berkeley.edu/Pubs/TechRpts/2020/EECS-2020-79.html). _E Liu_. EECS Department, UC Berkeley Technical Report, 2020. 


## License
FLOR is licensed under the [Apache v2 License](https://www.apache.org/licenses/LICENSE-2.0).
